## Словарь исправленных токенов

Итак, теперь наша задача — объединить функции, которые у нас уже есть, чтобы начать исправлять опечатки в тексте.

Позже, в интерфейсе нашего приложения, которое мы пишем, нам понадобится вывести текст, который уже будет содержать слова с исправленными опечатками и пометки о том, какие именно слова были исправлены.

Чтобы вывести этот текст, нам надо составить **словарь** специального вида. Ключом будет токен из входного текста, а значением к каждому токену — вложенный словарь.

### Вложенный словарь

Во-первых, мы ленивые, поэтому хотим делать как можно меньше работы, а значит, мы должны отсеять слова, которые не будем исправлять. Это части речи, которые функция `get_tokens_dict()` отмечает как `True`, то есть части речи, которые трудно или нельзя проанализировать на факт опечатки: служебные слова, аббревиатуры, спецсимволы и имена собственные. В нашем вложенном словаре за этот параметр будет отвечать ключ `'special'`. Если его значение `True`, это значит, что мы не работаем с токеном далее и выводим его таким, каким нашли, без исправлений и пометок. В случае `False` продолжаем работу.

Во-вторых, мы хотим запоминать, нашли ли мы в слове опечатку. За это будет отвечать ключ `'corrected'`, так же со значениями `True` и `False`. Если слово `'special': True`, логично тут сразу говорить, что опечаток мы в нем не нашли (потому что не искали). Если мы отметили слово как исправленное, то позже в тексте мы будем выводить его с соответсвующей пометкой.

В-третьих, мы должны знать, какой токен в итоге надо показать пользователю. Его мы будем хранить под ключом `'corrected_word'`.

Пример:
```
{'in': {
  'special': True,
  'corrected': False,
  'corrected_word': 'in'},
'letter': {
  'special': False,
  'corrected': False,
  'corrected_word': 'letter'},
'generql': {
  'special': False,
  'corrected': True,
  'corrected_word': 'general'},
}
```

Собственно, на данном этапе это все, что мы должны получить.



## Работа с JSON



Понятно, что мы будем исправлять опечатки сверяясь с каким-то источником, на который будем полагаться как на образец написания слов. В нашем случае это будет уже готовый для тебя набор данных. Но хранится он в отдельном файле и его надо как-то открыть...

А хранится он в файле типа JSON, то есть JavaScript Object Notation. Это файлы, которые позволяют сохранять объекты разных типов. Python умеет работать с этими файлами с помощью встроенной библиотеки json.

```
import json
```

Открыть какой-либо файл позволяет служебное слово `with` и функция `open()`.

Например, я хочу открыть произвольный текстовый файл с названием "1" и расширением txt, а обращаться буду к нему как к переменной file_1. Тогда:

```
with open('1.txt') as file_1:
  # после одного отступа осуществляется работа с файлом

# когда мы выходим из with, файл закрывается
```

Чтобы загрузить содержание файла в переменную, пользуются функцией load()

```
content = json.load()
```

В нашем файле-образце будет лежать только один массив (это соответствует питоновскому списку), поэтому в переменную запишется только один список, состоящий из слов.

In [ ]:
# Открой файл sample.json и запиши его содержание в переменную sample_list:


## Совмещение написанных ранее функций

### Инициализация `dict_of_corrections`

Теперь мы хотим пройти по тексту, который предложен пользователем, и создать словарь, который описали выше. Пусть он называется `dict_of_corrections`. Инициализируем его (в готовом для тебя виде и поясним код).



In [ ]:
text_tokens_dict = {
    'word': False,
    'USA': True,
    'applicqtion': False
}

dict_of_corrections = dict()
for key, value in text_tokens_dict.items():
    dict_of_corrections[key] = {
        'special': bool(), 'corrected': bool(), 'corrected_word': str()
    }

Перед началом работы мы предлагаем нашей функции какой-то небольшой словарь токенов `text_tokens_dict` (предположим, что он является результатом разбития на токены), содержанием которого мы заполним `dict_of_corrections`.

Теперь мы сообщаем, что переменная `dict_of_corrections` по типу является словарем.

Мы пополняем словарь токенами из исходного текста. Значит, мы обращаемся к функции `get_tokens_dict()`, которая нам возвращает словарь со значениями, является ли токен исправимым, и с помощью цикла пробегаемся по его значениям. Но как корректно пробегаться по словарю с помощью цикла? Тут очень полезен метод словаря `.items()`. Он превращает словарь в список из неизменяемых пар ключ-значение.

Чтобы с помощью цикла пробегаться по такому списку пар, мы можем пользоваться двумя переменными внутри оператора for. Смотри пример:

In [ ]:
example = [(1, 2), (3, 4), (5, 6)]

for key, value in example:
    print(key)
    print(value)

In [ ]:
# сделай тоже самое, но уже со словарем, с помощью .items():
example = {1: 2, 3: 4, 5: 6}

Точно так же мы поступим с нашим словарем. Мы хотим пройти по его значениями. Сперва мы будем добавлять соответствующий ключ в наш `dict_of_corrections`. Чтобы добавить или обновить значение в словаре `dict1` по ключу `key`, надо просто указать индекс: `dict1[key]`.

```
dict_of_corrections[token] = dict()
```
Но пока вложенных словарей у нас нет, мы хотим их заполнить тремя параметрами, озвученными нами в первом разделе (`special`, `corrected`, `corrected_word`). Мы делаем это, вставляя заготовку словаря, где указываем типы значений, которые будем хранить в них. `bool()` это булиан, то есть логическое `True` или `False`. `string()` возвращает просто пустую строчку. На этом инициализация закончена!



### Заполнение словаря

Во-первых, если функция `get_tokens_dict()` сопоставила токену значение `True`, то мы сразу знаем, что делать.

Заметь, что раз у нас в значении словаря вложен словарь, то чтобы обратиться **к индексу вложенного словаря**, достаточно написать **следующие друг за другом** индексы:

In [ ]:
if value == True:
    dict_of_corrections[key]['special'] = True  # Это функциональный токен?
    dict_of_corrections[key]['corrected'] = False
    dict_of_corrections[key]['corrected_word'] = key  # Что в итоге мы покажем? изменилось ли что-то?

Иной случай придется рассматривать более подробно.

```
else:
    # будем работать тут
```

Сначала, мы хотим узнать, есть ли в слове вообще опечатка. Если наше слово есть в `sample_list`, это значит, что мы примем его за правильно написанное. Условие вхождения проверяется с помощью служебного слова `in`.

Пример:



In [ ]:
print(1 in [1, 2, 'cat'])  # Есть ли элемент в списке?

print('me' in 'America')  # Есть ли подстрока в строке?

print('any money' in ['my purse', 'my life', 'my future'])  # Just for fun

Если мы начнем проверять слово `'House'` на вхождение в наш массив, оно не найдется! А все потому, что слова `'House'` и `'house'` отличаются начальным символом. Поэтому, когда мы хотим проверить вхождение слова в код, мы должны искать его версию без заглавных букв. Это делается методом .lower():


```
'HoUsE'.lower() >>> 'house'
```



In [ ]:
# Проверяем является ли слово специальным
if value == True:
    dict_of_corrections[key]['special'] = True
    dict_of_corrections[key]['corrected'] = False
    dict_of_corrections[key]['corrected_word'] = key
else:
    # Проверяем, есть ли слово в массиве образцовых слов.
    # Напиши свой код ниже
    # Воспользуйся методом .lower()



Остался один случай (опять `else:`). Наше слово не содержится в списке-образце. Значит, мы хотим его исправить. Чтобы это сделать, надо найти слово, которое по расстоянию Левенштейна ближе всего к нему.

Таких слов может оказаться несколько, но наш список-образец организован удобно — самые частые слова в нем ближе к началу, более редкие — в конце. Мы знаем еще одну вещь: если расстояние до какого-то слова оказалось ровно 1, **меньше уже не будет**.

То есть, в этом случае мы начинаем с помощью цикла проходить по значениям из sample_list и строить расстояния между ними и нашим токеном key. На каждом шаге цикла мы должны применить функцию `levenstein(str_1, str_2)`, и запоминать, до какого слова расстояние наименьшее и каково оно в точности.

Пусть мы будем записывать такое слово в переменную `correct_word`, а расстояние до него в `distance`. Только если расстояние оказывается **строго меньше**, тогда мы обновляем эти значения. Строго меньше потому, что каждое следующее слово в списке более редкое, а значит, меньше вероятность, что именно оно имелось в виду, поэтому **среди слов с одинаковым расстоянием** следует выбрать то, которое **раньше**.



In [ ]:
# Мы начнем работу ВНЕ ЦИКЛА, чтобы записать какое-то начальное значение distance, с которым можно будет сравнивать далее:
distance = levenstein(key.lower(), sample_list[0])
correct_word = sample_list[0]

# Определи для токена key то слово из sample_list, до которого расстояние наименьшее:

Если мы нашли слово с расстоянием 1, идти дальше уже нет смысла. Значит, цикл можно прервать. Это делается служебным словом `break`. Пример:

In [ ]:
list1 = [1, 2, 3, 4, 5, 6]

for number in list1:
  print(number)
  if number == 3:
    break

In [1]:
distance = levenstein(key, sample_list[0])
correct_word = sample_list[0]
# Определи для токена key слово из sample_list, до которого расстояние наименьшее


# Остановись, если нашел расстояние ровно 1:

В итоге, в случае, если мы корректировали слово, во вложенный `dict_of_corrections[key]` мы хотим сообщить, что **мы исправили слово**, и записать, **на что** мы его исправили.

In [ ]:
distance = levenstein(key, sample_list[0])
correct_word = sample_list[0]
# Определи для токена key слово из sample_list, до которого расстояние наименьшее
# Остановись, если нашел расстояние ровно 1

# Когда мы закончили работу в цикле, заполни словарь:

Осталось объединить всё в функцию build_dict(). Перед этим нужно подготовить функции для создания массива образцовых слов и словаря токенов из текста, а также для рассчёта расстояния Левенштейна. Первое тебе нужно сделать самому, остальное мы подготовили и осталось лишь запустить:

In [ ]:
import json  # Импортируем библиотеку для открытия массива с образцовыми словами


def get_words_massive():
  # Открой здесь json массив с образцовыми словами
  # и присвой его переменной words_massive

  # Возвращаем массив образцовых слов
  return words_massive

In [ ]:
# Запусти эту ячейку, чтобы создать функцию по получению словаря токенов.
# Функция является просто образцом вывода на основе предложенного текста
def get_tokens_dict(text):
    tokens_dict = {
        '\n': True,
        'Twin': True,
        'Peaks': True,
        'is': False,
        'an': False,
        'Amercan': False,
        'mystery': False,
        'sirial': False,
        'drama': False,
        'televizion': False,
        'series': False,
        'created': False,
        'by': True,
        'Mark': True,
        'Frost': True,
        'and': True,
        'David': True,
        'Lynch': True,
        '.': True,
        'It': False,
        'premeired': False,
        'on': True,
        'ABC': True,
        'April': True,
        '8': False,
        ',': True,
        '1990': False,
        'ran': False,
        'for': True,
        'twoo': False,
        'seasons': False,
        'until': True,
        'its': False,
        'cancellation': False,
        'in': True,
        '1991': False,
        'The': False,
        'show': False,
        'returned': False,
        '2017': False,
        'a': False,
        'third': False,
        'season': False,
        'Showtime': True
    }

    return tokens_dict

In [ ]:
# Запусти эту ячейку, чтобы создать функцию по получению расстояния Левенштейна
def levenstein(str_1, str_2):
    n = len(str_1)
    m = len(str_2)
    current_row = []
    for k in range(m+1):
        current_row += [k]
    for i in range(1, n+1):
        previous_row = current_row
        current_row = [i] + [0] * m
        for j in range(1, m+1):
            left = current_row[j-1] + 1
            up = previous_row[j] + 1
            left_up = previous_row[j-1]
            if str_1[i-1] != str_2[j-1]:
                left_up += 1
        current_row[j] = min(up, left, left_up)
    return current_row[-1]

Наконец, заполни функцию `build_dict(text)`, следуя инструкции ниже:

1.   Открыть json файл и записать его содержимое в `sample_list`
2.   Инициализировать словарь `dict_of_corrections`
3.   Применить на тексте `text` функцию `get_tokens_dict()`
4.   Пройти по парам `key` `value` из ее результа, пользуясь `.items()`
5.   Проверить, токен `key` попадает в категорию `'special'`
6.   Если нет, то проверить, он входит в список `sample_list`
7.   Если опять нет, то исправить его и запомнить, на что мы его исправили
8.   Вернуть заполненный словарь `dict_of_corrections`

In [ ]:
def build_dict(text):
    words_massive = get_words_massive()  # Получим массив образцовых слов
    text_tokens_dict = get_tokens_dict(text)  # Получим словарь токенов из текста

    # Иницилизируем словарь, который заполним далее:


    # Напиши код, который заполнит словарь dict_of_corrections:


    # Верни этот словарь
    return dict_of_corrections


In [1]:
# Запусти эту ячейку, чтобы проверить, всё ли работает
text = """
Twin Peaks is an American mystery sirial drama televizion series
created by Mark Frost and David Lynch. It premeired on ABC on April 8, 1990,
and ran for twoo seasons until its cancellation in 1991. The show returned
in 2017 for a third season on Showtime.
"""

build_dict(text)